<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#написать-на-pytorch-глубокую-сеть,-проверить-работу-форвардпасса." data-toc-modified-id="написать-на-pytorch-глубокую-сеть,-проверить-работу-форвардпасса.-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>написать на pytorch глубокую сеть, проверить работу форвардпасса.</a></span></li><li><span><a href="#написать-адаптивный-оптимизатор" data-toc-modified-id="написать-адаптивный-оптимизатор-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>написать адаптивный оптимизатор</a></span></li><li><span><a href="#решить-задачу-нахождения-корней-квадратного-уравнения-методом-градиентнго-спуска" data-toc-modified-id="решить-задачу-нахождения-корней-квадратного-уравнения-методом-градиентнго-спуска-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>решить задачу нахождения корней квадратного уравнения методом градиентнго спуска</a></span></li></ul></div>

Школа DataScientist, декабрь 2022, Андреева Ольга

# написать на pytorch глубокую сеть, проверить работу форвардпасса.

In [49]:
import torch
import torch.nn as nn
import numpy as np

In [50]:
class SomeDeepNN(nn.Module):
    def __init__(self,):
        super().__init__()
        self.layers=[
            {'model':nn.Linear(3, 5), 'activation':nn.ReLU()}
            ,{'model':nn.BatchNorm1d(5, 5), 'activation':nn.ReLU()}
            ,{'model':nn.Linear(5, 8), 'activation':nn.ReLU()}
            ,{'model':nn.Linear(8, 8), 'activation':nn.Sigmoid()}
            ,{'model':nn.BatchNorm1d(8, 8), 'activation':None}
            ,{'model':nn.Linear(8, 8), 'activation':nn.ReLU()}
            ,{'model':nn.Dropout(0.5), 'activation':None}
            ,{'model':nn.BatchNorm1d(8, 8), 'activation':None}
            ,{'model':nn.Linear(8, 3), 'activation':None}
        ]
      
    def forward (self, x):
        for layer in self.layers:
            x = layer['model'](x)
            x = layer['activation'](x) if layer['activation']!= None else x
        return x
    


In [51]:
model=SomeDeepNN()
model.train()

SomeDeepNN()

In [52]:
batch=torch.randn(10, 3)*100
batch

tensor([[ 188.6986,  -42.3589,   50.8815],
        [ -15.7684,    1.7608, -122.7787],
        [  29.9381,   92.1885,  117.2309],
        [  40.6160, -140.8805, -129.0985],
        [ 197.3719,  -45.7335,   70.3666],
        [  98.4731,  -33.6356,  -70.3143],
        [ -47.7658,   92.0339,   39.7146],
        [ 146.4833,  126.3035,  -51.2536],
        [ -44.3214,  123.7859,    2.5476],
        [-271.6638,   47.1077,   31.3210]])

In [53]:
model(batch)

tensor([[-0.0105, -0.1834, -0.1137],
        [-0.0073, -0.1855, -0.1114],
        [-0.0331, -0.1867, -0.1193],
        [-0.0052, -0.1868, -0.1098],
        [-0.0076, -0.1852, -0.1116],
        [-0.0238, -0.1834, -0.1131],
        [-0.0279, -0.1810, -0.1157],
        [-0.0247, -0.1833, -0.1133],
        [-0.0336, -0.1868, -0.1194],
        [-0.0277, -0.1813, -0.1158]], grad_fn=<AddmmBackward0>)

# написать адаптивный оптимизатор

In [54]:
def sigmoid(x):
    return 1. / (1 + np.exp(-x))

def sigmoid_backward(da, x):
    sig = sigmoid(x)
    
    return da * sig * (1 - sig)

def relu(x):
    return np.maximum(0., x)

def relu_backward(da, x):
    da = np.array(da, copy = True)
    da[x <= 0] = 0;
    return da;
def mse_loss(t, y):
    return (t - y) ** 2

def d_mse_loss(t, y):
    return 2 * (y - t) 


In [55]:
class LinearLayer:
    def __init__(self, n_inp, n_out, activation='sigmoid'):
        self.w = np.random.randn(n_out, n_inp) * 0.1 #weigth
        self.b = np.random.randn(n_out, 1) * 0.1     #bias
        if activation == 'sigmoid':
            self.activ = sigmoid
        if activation == 'relu':
            self.activ = relu
        elif activation == 'None':
            self.activ = None
        else:
            raise Exception(f'Unknown activation "{activation}"')
        self._clear_state()

    def _clear_state(self):
        self.lin = None
        self.inp = None
        self.d_w = None
        self.d_b = None

    def forward(self, x):
        self.inp = x
        self.lin = np.dot(self.w, x) + self.b
        activ = self.activ(self.lin) if self.activ is not None else self.lin

        return activ

    def backward(self, grad): # grad = d L / d z    Dout 
        # grad * dz / d lin
        if self.activ == sigmoid:
            grad_lin = sigmoid_backward(grad, self.lin) 
        elif self.activ == relu:
            grad_lin = relu_backward(grad, self.lin)
        else:
            grad_lin = grad
        # grad_lin * d lin / d w 
        m = self.inp.shape[1]
        self.d_w = np.dot(grad_lin, self.inp.T) / m    # d_in dOut
        # grad_lin * d lin / d b 
        self.d_b = np.sum(grad_lin, axis=1, keepdims=True) / m

        grad = np.dot(self.w.T, grad_lin)

        return grad


In [56]:
from typing import Tuple

class Model:
    def __init__(self, arch: Tuple[Tuple[int, int]], activation):
        self.layers = []
        for i, p in enumerate(arch):
            self.layers.append(
                LinearLayer(p[0], p[1], 
                            activation=activation if i < len(arch)-1 else 'None')
                )
        self._clear_state()
    
    def _clear_state(self):
        for l in self.layers:
            l._clear_state()

    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def backward(self, grad):
        for layer in reversed(self.layers):
            grad = layer.backward(grad)
        return grad 

In [57]:
#для модели
class RMSProp:
    def __init__(self, model: LinearLayer, rho=0.99, lr=0.001):
        self.lr = lr
        self.rho = rho
        self.model = model
        self.vel_w=[]
        self.vel_b=[]
        for layer in self.model.layers:
            self.vel_w.append(np.zeros_like(layer.w))
            self.vel_b.append(np.zeros_like(layer.b))
        
    def step(self):
        for i, layer in enumerate(self.model.layers):
            
            self.vel_w[i] = self.rho * self.vel_w[i] + (1 - self.rho) * layer.d_w**2
            self.vel_b[i] = self.rho * self.vel_b[i] + (1 - self.rho) * layer.d_b**2
            
            layer.w -= layer.d_w * self.lr/(self.vel_w[i]**0/5)
            layer.b -= layer.d_b * self.lr/(self.vel_b[i]**0/5)

    def zero_grad(self):
        for layer in self.model.layers:
            layer.d_w = np.zeros_like(layer.d_w)
            layer.d_b = np.zeros_like(layer.d_b)
    
            

In [58]:
x = np.random.uniform(-2, 2, 20000)
y = x**2 + np.random.randn()*0.01
print(x)
print(y)

[ 1.03799105  1.87060634 -0.68860144 ...  0.2346453   0.03657749
  1.67475985]
[1.08881509 3.51055773 0.48556161 ... 0.06644808 0.01272758 2.81621021]


In [59]:
model = Model(((1, 100), (100, 1)), activation='relu')
optim = RMSProp(model)
for e in range(20):
    for i, (val, t) in enumerate(zip(x, y)):
        optim.zero_grad()
        pred = model.forward(np.array([[val]]))
        loss = mse_loss(t, pred)
        grad = d_mse_loss(t, pred)
        model.backward(grad)
        optim.step()
    print(e, model.forward([[1]]), model.forward([[2]]), model.forward([[-1]]), model.forward([[-0.5]]))       

0 [[0.98517878]] [[3.86524164]] [[0.99044424]] [[0.26625722]]
1 [[1.0069398]] [[3.92873267]] [[1.00440806]] [[0.25963116]]
2 [[1.01451602]] [[3.95391384]] [[1.0144683]] [[0.26227908]]
3 [[1.01790828]] [[3.96678813]] [[1.01274495]] [[0.26277565]]
4 [[1.01923901]] [[3.9744744]] [[1.01084439]] [[0.26261078]]
5 [[1.02056346]] [[3.97971381]] [[1.00947312]] [[0.26230627]]
6 [[1.02082732]] [[3.98307646]] [[1.00842799]] [[0.26203254]]
7 [[1.02131839]] [[3.98576036]] [[1.00755432]] [[0.26176604]]
8 [[1.02157074]] [[3.98776906]] [[1.00668923]] [[0.26153397]]
9 [[1.02190264]] [[3.98953573]] [[1.00576562]] [[0.26137117]]
10 [[1.02194349]] [[3.9908611]] [[1.00507554]] [[0.26119673]]
11 [[1.02220317]] [[3.99189321]] [[1.00434805]] [[0.26109218]]
12 [[1.02215024]] [[3.99291003]] [[1.00372964]] [[0.26096549]]
13 [[1.02233486]] [[3.99400428]] [[1.00314233]] [[0.26081377]]
14 [[1.02200045]] [[3.99501536]] [[1.00259559]] [[0.26083529]]
15 [[1.02222793]] [[3.99592813]] [[1.00196812]] [[0.26100221]]
16 [[1

# решить задачу нахождения корней квадратного уравнения методом градиентнго спуска

In [60]:
# Task 3
# Find the roots of square equation by gradient descent
# x ** 2 - 6 * x + 4 = 0


# посчитать производную от преобразованной функции
# надо начать движение от начальной точки в направлении антиградиента с заданным шагом
# x = x - lr * grad(x)
# всегда ли сойдемся за приемлемое количество шагов?
    # Нет. Не сойдемся, еслм неудачно выбрали начальную точку и lr. Ну или у уравнения может не быть корней.
# важна ли начальная точка?
    # Важна. 
    # Если инициализируемся в максимуме, grad(x)=0, x не меняется, вообще никуда не придём.
    # Если начальная точка далеко от корня, grad(x) -> inf.
        # При большом lr в этом случае мы на каждом шаге пролетаем мимо точки минимума и удаляемся от нее.
        # При маленьком lr в этом случае улетает в небеса число шагов, за которое находим корень.    
# как найти второй корень?
    # Инициализироваться в другой точке.
    # В случае с квадратным уравнением можно найти вершину параболы и выбрать новую точку отсчета относительно вершины.
# как вляет ЛР?
    # Если lr очень большое, можно никогда не попасть в точку минимума,
    # т.к. будут слишком большие шаги, и от искомой точки мы будем удаляться, а не приближаться.
    # с уменьшением lr возрастает точность попадания, но растет число шагов, и, соответственно, время, за которое находим корень  

In [61]:
class equation:
    def __init__ (self, cfs):
        self.cfs=np.array(cfs) #коэффициенты квадратного уравнения
        self.plnm=np.poly1d(cfs)
        self.sq_plnm=np.polymul(self.plnm, self.plnm)  #функция, которую будем минимизировать - квадрат значения многочлена

    def func(self, p_plnm, x): #резульат функции, которую будем минимизировать - квадрат значения многочлена
        return (np.polyval(p_plnm, x))

    def func_deriv(self, p_plnm, x): #прозводная функции, которую будем минимизировать
        return np.polyval(np.polyder(p_plnm, m=1),x)

    def gd(self, p_plnm, init_x, lr, steps, precision): #градиентный спуск
        x=init_x
        for step in range(steps):
            df = self.func_deriv(p_plnm, x)
#            if step%1000==0:
#                print (step, x, df)
            if abs(df) < precision:
                if self.func(p_plnm, x)<self.func(p_plnm, x-precision): #проверка, что не попали в максимум
                    print('found in', step, 'steps')
                    return (x)
            x = x - lr*df
 
        if step >= steps-1:
            print (step, x, df)
            print('not found any root in', steps, 'steps')
            return (None)


In [62]:
%%time
sq_eq = equation([1, -6, 4])
x=sq_eq.gd(sq_eq.sq_plnm, 1000, 1e-7, 10000000, 0.001)
print('x =', x)    

found in 2677045 steps
x = 5.236092977043281
Wall time: 2min 56s


In [63]:
%%time
##top=-sq_eq.cfs[1]/2*sq_eq.cfs[0]
top=sq_eq.gd(sq_eq.plnm, 0, 0.001, 10000, 0.01)


x1=sq_eq.gd(sq_eq.sq_plnm, top-1, 0.0001, 10000, 0.001)
x2=sq_eq.gd(sq_eq.sq_plnm, top+1, 0.0001, 10000, 0.001)
print('x1 =', x1)
print('x2 =', x2)

found in 3196 steps
found in 3016 steps
found in 3022 steps
x1 = 0.7639569884992341
x2 = 5.236042987137295
Wall time: 625 ms
